In [1]:
# Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
        

def main():
    database = "sqlite_db_pythonsqlite.db" 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


### Questions and Solutions from 1 to 9

Here are solutions 1 through 9 for convenient reference:

/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

    SELECT name FROM Facilities WHERE membercost != 0

/* Q2: How many facilities do not charge a fee to members? */

    SELECT COUNT(*) FROM Facilities WHERE membercost = 0

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */
    SELECT facid, name, membercost, monthlymaintenance
    FROM Facilities
    WHERE membercost > 0.0 AND membercost < monthlymaintenance * 0.2

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

    SELECT * FROM Facilities WHERE facid IN ( 1,5 )

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

    SELECT name, monthlymaintenance,
    CASE 
        WHEN monthlymaintenance > 100 THEN 'Expensive'
        ELSE 'Cheap'
    END AS facility_label
    FROM Facilities;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

    SELECT firstname, surname, joindate
    FROM Members
    ORDER BY joindate DESC

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

    SELECT DISTINCT f.name AS court, CONCAT(m.firstname, ' ', m.surname) AS member 
    FROM Members AS m 
    INNER JOIN Bookings AS b ON (m.memid = b.memid) 
    INNER JOIN Facilities AS f ON (b.facid = f.facid) 
    WHERE f.name LIKE 'Tennis%' 
    ORDER BY member;

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

    SELECT f.name AS facility, CONCAT(m.firstname, ' ', m.surname) AS member, (CASE WHEN m.memid=0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS cost 
    FROM Members AS m 
    INNER JOIN Bookings AS b ON (m.memid=b.memid) 
    INNER JOIN Facilities AS f ON (b.facid=f.facid) 
    WHERE date(b.starttime) = '2012-09-14'
    HAVING cost > 30
    ORDER BY cost DESC;

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

    SELECT member, facility, cost 
    FROM 
        (SELECT f.name as facility, CONCAT(m.firstname,' ',m.surname) AS member, 
         (CASE WHEN m.memid = 0 THEN b.slots * f.guestcost ELSE b.slots * f.membercost END) AS cost
         FROM Members AS m 
         INNER JOIN Bookings AS b ON m.memid = b.memid 
         INNER JOIN Facilities AS f ON b.facid = f.facid 
         WHERE DATE(b.starttime) = '2012-09-14'
        ) AS sub
    WHERE cost > 30
    ORDER BY cost DESC;


In [2]:
def Q10(conn):

    cur = conn.cursor()    
    query10 = """
        SELECT subquery.name, subquery.revenue
        FROM
            (SELECT f.name, 
             SUM( CASE WHEN b.memid =0 THEN b.slots * f.guestcost
                      ELSE b.slots * f.membercost END ) AS revenue
             FROM `Facilities` AS f
                 INNER JOIN `Bookings` AS b 
                 ON f.facid = b.facid
        GROUP BY f.name) AS subquery

        WHERE subquery.revenue < 1000
        ORDER BY subquery.revenue
        """
    cur.execute(query10)
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main_Q10():
    database = "sqlite_db_pythonsqlite.db" 
    conn = create_connection(database)
    with conn: 
        print("Q10: List of facilities with a total revenue less than 1000.")
        Q10(conn)
 
if __name__ == '__main__':
    main_Q10()

2.6.0
Q10: List of facilities with a total revenue less than 1000.
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)


In [3]:
def Q11(conn):

    cur = conn.cursor()  
    query11 = """
        SELECT m.memid, m.surname, m.firstname, (r.firstname || ' ' || r.surname) AS recommender
        FROM `Members` AS m
        INNER JOIN `Members` AS r
        ON m.recommendedby = r.memid
        WHERE m.memid != 0
        ORDER BY m.surname, m.firstname
        """
    cur.execute(query11)
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main_Q11():
    database = "sqlite_db_pythonsqlite.db" 
    conn = create_connection(database)
    with conn: 
        print("Q11: Report of members and who recommended them in alphabetic surname,firstname order")
        Q11(conn)
 
if __name__ == '__main__':
    main_Q11()

2.6.0
Q11: Report of members and who recommended them in alphabetic surname,firstname order
(15, 'Bader', 'Florence', 'Ponder Stibbons')
(12, 'Baker', 'Anne', 'Ponder Stibbons')
(16, 'Baker', 'Timothy', 'Jemima Farrell')
(8, 'Boothe', 'Tim', 'Tim Rownam')
(5, 'Butters', 'Gerald', 'Darren Smith')
(22, 'Coplin', 'Joan', 'Timothy Baker')
(36, 'Crumpet', 'Erica', 'Tracy Smith')
(7, 'Dare', 'Nancy', 'Janice Joplette')
(20, 'Genting', 'Matthew', 'Gerald Butters')
(35, 'Hunt', 'John', 'Millicent Purview')
(11, 'Jones', 'David', 'Janice Joplette')
(26, 'Jones', 'Douglas', 'David Jones')
(4, 'Joplette', 'Janice', 'Darren Smith')
(21, 'Mackenzie', 'Anna', 'Darren Smith')
(10, 'Owen', 'Charles', 'Darren Smith')
(17, 'Pinker', 'David', 'Jemima Farrell')
(30, 'Purview', 'Millicent', 'Tracy Smith')
(27, 'Rumney', 'Henrietta', 'Matthew Genting')
(24, 'Sarwin', 'Ramnaresh', 'Florence Bader')
(14, 'Smith', 'Jack', 'Darren Smith')
(9, 'Stibbons', 'Ponder', 'Burton Tracy')
(29, 'Worthington-Smyth', 'Henr

In [4]:
def Q12(conn):

    cur = conn.cursor()   
    query12 = """
        SELECT b.facid, f.name, SUM(b.slots) AS usage_by_member
        FROM `Bookings` AS b
        INNER JOIN `Facilities` AS f ON b.facid = f.facid
        WHERE b.memid !=0
        GROUP BY b.facid
        """
    cur.execute(query12)
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def main_Q12():
    database = "sqlite_db_pythonsqlite.db" 
    conn = create_connection(database)
    with conn: 
        print("Q12: Facilities with their usage by member, but not guests")
        Q12(conn)
  
if __name__ == '__main__':
    main_Q12()

2.6.0
Q12: Facilities with their usage by member, but not guests
(0, 'Tennis Court 1', 957)
(1, 'Tennis Court 2', 882)
(2, 'Badminton Court', 1086)
(3, 'Table Tennis', 794)
(4, 'Massage Room 1', 884)
(5, 'Massage Room 2', 54)
(6, 'Squash Court', 418)
(7, 'Snooker Table', 860)
(8, 'Pool Table', 856)


In [5]:
def Q13(conn):

    cur = conn.cursor()   
    query13 = """
        SELECT b.facid, f.name, strftime('%m', b.starttime) as month, SUM(b.slots) AS usage_by_month
        FROM `Bookings` AS b
        INNER JOIN `Facilities` AS f 
        ON b.facid = f.facid
        WHERE b.memid !=0
        GROUP BY b.facid, month
        """
    cur.execute(query13)
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main_Q13():
    database = "sqlite_db_pythonsqlite.db" 
    conn = create_connection(database)
    with conn: 
        print("Q13: Facilities usage by month, but not guests")
        Q13(conn)
 
if __name__ == '__main__':
    main_Q13()

2.6.0
Q13: Facilities usage by month, but not guests
(0, 'Tennis Court 1', '07', 201)
(0, 'Tennis Court 1', '08', 339)
(0, 'Tennis Court 1', '09', 417)
(1, 'Tennis Court 2', '07', 123)
(1, 'Tennis Court 2', '08', 345)
(1, 'Tennis Court 2', '09', 414)
(2, 'Badminton Court', '07', 165)
(2, 'Badminton Court', '08', 414)
(2, 'Badminton Court', '09', 507)
(3, 'Table Tennis', '07', 98)
(3, 'Table Tennis', '08', 296)
(3, 'Table Tennis', '09', 400)
(4, 'Massage Room 1', '07', 166)
(4, 'Massage Room 1', '08', 316)
(4, 'Massage Room 1', '09', 402)
(5, 'Massage Room 2', '07', 8)
(5, 'Massage Room 2', '08', 18)
(5, 'Massage Room 2', '09', 28)
(6, 'Squash Court', '07', 50)
(6, 'Squash Court', '08', 184)
(6, 'Squash Court', '09', 184)
(7, 'Snooker Table', '07', 140)
(7, 'Snooker Table', '08', 316)
(7, 'Snooker Table', '09', 404)
(8, 'Pool Table', '07', 110)
(8, 'Pool Table', '08', 303)
(8, 'Pool Table', '09', 443)
